# script to do experiments described in paper: Deep Interest Evolution Network for Click-Through Rate Prediction

## how to run

1. Please download data.tar.gz, data1.tar.gz and data2.tar.gz from https://github.com/mouna99/dien and decompress them to the same folder with this script.

2. Please run prepare_neg.ipynb first.

In [ ]:
SEQ_MAX_LEN = 100 # maximum sequence length
BATCH_SIZE = 128
EMBEDDING_DIM = 18
DNN_HIDDEN_SIZE = [200, 80]
DNN_DROPOUT = 0.0
TEST_RUN = False
EPOCH = 2
SEED = 10

In [ ]:
%matplotlib inline

import itertools
from collections import Counter, OrderedDict

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score

from prediction_flow.features import Number, Category, Sequence, Features
from prediction_flow.transformers.column import (
    StandardScaler, CategoryEncoder, SequenceEncoder)

from prediction_flow.pytorch.data import Dataset
from prediction_flow.pytorch import WideDeep, DeepFM, DNN, DIN, DIEN, AttentionGroup

from prediction_flow.pytorch.functions import fit, predict, create_dataloader_fn

In [ ]:
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [ ]:
train_df = pd.read_csv(
    "./local_train.csv", sep='\t')

valid_df = pd.read_csv(
    "./local_test.csv", sep='\t')

In [ ]:
if TEST_RUN:
    train_df = train_df.sample(1000)
    valid_df = valid_df.sample(1000)

In [ ]:
train_df.head()

In [ ]:
valid_df.head()

# EDA

In [ ]:
def scale_eda(df):
    print(df.shape)
    print(df.uid.nunique())
    print(df.mid.nunique())
    print(df.groupby('label', as_index=False).uid.count())

In [ ]:
scale_eda(train_df)
scale_eda(valid_df)

In [ ]:
train_df.values[0][4].split('\x02')

**This data set is well balanced. Each user has two samples, pos sample and neg sample.**

In [ ]:
unique_cats = Counter(train_df.cat.values.tolist())

In [ ]:
unique_cats_in_hist = Counter(
    itertools.chain(*train_df.hist_cats.apply(lambda x: x.split("")).values.tolist()))

In [ ]:
print(len(unique_cats), len(unique_cats_in_hist),
      len(np.intersect1d(list(unique_cats.keys()), list(unique_cats_in_hist.keys()))))

**All categorys also appear in history categorys.**

In [ ]:
unique_mids = Counter(train_df.mid.values.tolist())

In [ ]:
unique_mids_in_hist = Counter(
    itertools.chain(*train_df.hist_mids.apply(lambda x: x.split("")).values.tolist()))

In [ ]:
print(len(unique_mids), len(unique_mids_in_hist),
      len(np.intersect1d(list(unique_mids.keys()), list(unique_mids_in_hist.keys()))))

**Most mids appears in history mids.**

In [ ]:
print("There are {}% mid overlap between train and valid".format(
    100 * len(np.intersect1d(train_df.mid.unique(), valid_df.mid.unique())) / len(valid_df.mid.unique())))

In [ ]:
print("There are {}% mid overlap between train and valid".format(
    100 * len(np.intersect1d(train_df.cat.unique(), valid_df.cat.unique())) / len(valid_df.cat.unique())))

# define features

In [ ]:
cat_enc = SequenceEncoder(sep="\x02", min_cnt=1, max_len=SEQ_MAX_LEN)

In [ ]:
cat_enc.fit(train_df.hist_cats.values)

In [ ]:
cat_word2idx, cat_idx2word = cat_enc.word2idx, cat_enc.idx2word

In [ ]:
print(len(cat_word2idx))

In [ ]:
mid_enc = SequenceEncoder(sep="\x02", min_cnt=1, max_len=SEQ_MAX_LEN)

In [ ]:
mid_enc.fit(np.vstack([train_df.mid.values, train_df.hist_mids.values]))

In [ ]:
mid_word2idx, mid_idx2word = mid_enc.word2idx, mid_enc.idx2word

In [ ]:
print(len(mid_word2idx))

In [ ]:
number_features = []

category_features = [
    Category('mid',
             CategoryEncoder(min_cnt=1, word2idx=mid_word2idx, idx2word=mid_idx2word),
             embedding_name='mid'),
    Category('cat',
             CategoryEncoder(min_cnt=1, word2idx=cat_word2idx, idx2word=cat_idx2word),
             embedding_name='cat'),
]

sequence_features = [
    Sequence('hist_mids',
             SequenceEncoder(sep="\x02", min_cnt=1, max_len=SEQ_MAX_LEN,
                             word2idx=mid_word2idx, idx2word=mid_idx2word),
             embedding_name='mid'),
    Sequence('hist_cats',
             SequenceEncoder(sep="\x02", min_cnt=1, max_len=SEQ_MAX_LEN,
                             word2idx=cat_word2idx, idx2word=cat_idx2word),
             embedding_name='cat')
]

features, train_loader, valid_loader = create_dataloader_fn(
    number_features, category_features, sequence_features, BATCH_SIZE, train_df, 'label', valid_df, 4)

In [ ]:
def evaluation(model, df, dataloader):
    preds = predict(model, dataloader)
    return roc_auc_score(df['label'], preds.ravel())

In [ ]:
din_attention_groups = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'mid', 'pos_hist': 'hist_mids'},
               {'ad': 'cat', 'pos_hist': 'hist_cats'}],
        hidden_layers=[80, 40], att_dropout=0.0)]

gru_attention_groups = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'mid', 'pos_hist': 'hist_mids'},
               {'ad': 'cat', 'pos_hist': 'hist_cats'}],
        hidden_layers=[80, 40], att_dropout=0.0, gru_type='GRU')]

aigru_attention_groups = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'mid', 'pos_hist': 'hist_mids'},
               {'ad': 'cat', 'pos_hist': 'hist_cats'}],
        hidden_layers=[80, 40], att_dropout=0.0, gru_type='AIGRU')]

agru_attention_groups = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'mid', 'pos_hist': 'hist_mids'},
               {'ad': 'cat', 'pos_hist': 'hist_cats'}],
        hidden_layers=[80, 40], att_dropout=0.0, gru_type='AGRU')]

augru_attention_groups = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'mid', 'pos_hist': 'hist_mids'},
               {'ad': 'cat', 'pos_hist': 'hist_cats'}],
        hidden_layers=[80, 40], att_dropout=0.0, gru_type='AUGRU')]

models = [
    ('DNN', DNN(features, 2, EMBEDDING_DIM, DNN_HIDDEN_SIZE,
        final_activation='sigmoid', dropout=DNN_DROPOUT)),
    
    ('WideDeep', WideDeep(features,
             wide_features=['mid', 'hist_mids', 'cat', 'hist_cats'],
             deep_features=['mid', 'hist_mids', 'cat', 'hist_cats'],
             cross_features=[('mid', 'hist_mids'), ('cat', 'hist_cats')],
             num_classes=2, embedding_size=EMBEDDING_DIM, hidden_layers=DNN_HIDDEN_SIZE,
             final_activation='sigmoid', dropout=DNN_DROPOUT)),
    
    ('DeepFM', DeepFM(features, 2, EMBEDDING_DIM, DNN_HIDDEN_SIZE, 
           final_activation='sigmoid', dropout=DNN_DROPOUT)),
    
    ('DIN', DIN(features, din_attention_groups, 2, EMBEDDING_DIM, DNN_HIDDEN_SIZE,
        final_activation='sigmoid', dropout=DNN_DROPOUT)),
    
    ('DIEN_gru', DIEN(features, gru_attention_groups, 2, EMBEDDING_DIM, DNN_HIDDEN_SIZE,
         final_activation='sigmoid', dropout=DNN_DROPOUT)),
    
    ('DIEN_aigru', DIEN(features, aigru_attention_groups, 2, EMBEDDING_DIM, DNN_HIDDEN_SIZE,
         final_activation='sigmoid', dropout=DNN_DROPOUT)),
    
    ('DIEN_agru', DIEN(features, agru_attention_groups, 2, EMBEDDING_DIM, DNN_HIDDEN_SIZE,
         final_activation='sigmoid', dropout=DNN_DROPOUT)),
    
    ('DIEN_augru', DIEN(features, augru_attention_groups, 2, EMBEDDING_DIM, DNN_HIDDEN_SIZE,
         final_activation='sigmoid', dropout=DNN_DROPOUT))
]

In [ ]:
def run(models):
    scores = OrderedDict()
    model_loss_curves = OrderedDict()
    for model_name, model in models:
        print(model_name)
        loss_func = nn.BCELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer=optimizer, mode='min', factor=0.5, patience=5)
    
        losses = fit(EPOCH, model, loss_func, optimizer,
            train_loader, valid_loader, scheduler, notebook=True, auxiliary_loss_rate=1)
    
        scores[model_name] = evaluation(model, valid_df, valid_loader)
        model_loss_curves[model_name] = losses
    return scores, model_loss_curves

In [ ]:
scores1, model_loss_curves1 = run(models)

In [ ]:
print(scores1)

In [ ]:
print(model_loss_curves1)

In [ ]:
number_features = []

category_features = [
    Category('mid',
             CategoryEncoder(min_cnt=1, word2idx=mid_word2idx, idx2word=mid_idx2word),
             embedding_name='mid'),
    Category('cat',
             CategoryEncoder(min_cnt=1, word2idx=cat_word2idx, idx2word=cat_idx2word),
             embedding_name='cat'),
]

sequence_features = [
    Sequence('hist_mids',
             SequenceEncoder(sep="", min_cnt=1, max_len=SEQ_MAX_LEN,
                             word2idx=mid_word2idx, idx2word=mid_idx2word),
             embedding_name='mid'),
    Sequence('hist_cats',
             SequenceEncoder(sep="", min_cnt=1, max_len=SEQ_MAX_LEN,
                             word2idx=cat_word2idx, idx2word=cat_idx2word),
             embedding_name='cat'),
    Sequence('neg_hist_mids',
             SequenceEncoder(sep="", min_cnt=1, max_len=SEQ_MAX_LEN,
                             word2idx=mid_word2idx, idx2word=mid_idx2word),
             embedding_name='mid'),
    Sequence('neg_hist_cats',
             SequenceEncoder(sep="", min_cnt=1, max_len=SEQ_MAX_LEN,
                             word2idx=cat_word2idx, idx2word=cat_idx2word),
             embedding_name='cat')
]

features, train_loader, valid_loader = create_dataloader_fn(
    number_features, category_features, sequence_features, BATCH_SIZE, train_df, 'label', valid_df, 4)

In [ ]:
augru_attention_groups_with_neg = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'mid', 'pos_hist': 'hist_mids', 'neg_hist': 'neg_hist_mids'},
               {'ad': 'cat', 'pos_hist': 'hist_cats', 'neg_hist': 'neg_hist_cats'}],
        hidden_layers=[80, 40], att_dropout=0.0, gru_type='AUGRU')]

In [ ]:
models = [
    ('DIEN', DIEN(features, augru_attention_groups_with_neg, 2, EMBEDDING_DIM, DNN_HIDDEN_SIZE,
         final_activation='sigmoid', dropout=DNN_DROPOUT, use_negsampling=True))
]

In [ ]:
scores2, model_loss_curves2 = run(models)

In [ ]:
print(scores2)

In [ ]:
print(model_loss_curves2)

In [ ]:
scores = scores1.copy()

In [ ]:
scores['DIEN'] = scores2['DIEN']

In [ ]:
scores

In [ ]:
def autolabel(rects):
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,
                '%f' % height,
                ha='center', va='bottom')

In [ ]:
_, ax = plt.subplots(figsize=(10, 5))
rect = ax.bar(list(scores.keys()), list(scores.values()))
ax.set_ylabel('AUC')
ax.set_ylim(top=0.9)
autolabel(rect)
plt.plot()